In [3]:
import os
import torch
from tqdm import tqdm
import pandas as pd

In [4]:
DATA_ROOT = '../../dataset'
TEST_DATASET_PATH = os.path.join(DATA_ROOT, 'test')

INFERENCE_SIZE = 1280
AUGMENT = True

IMAGES = []
for image_name in os.listdir(TEST_DATASET_PATH):
    if not image_name.startswith('._'):
        IMAGES.append(os.path.join(TEST_DATASET_PATH, image_name))

IMAGES.sort()

In [5]:
project = 'bc_lv2-object_detection'
name = 'yolov5l6_pretrained_highaug_imgsize1280_pseudolabeling_HONEY'

In [6]:
model = torch.hub.load('./', 'custom', path = f'{project}/{name}/weights/best.pt', source = 'local')
model.conf = 0.001  # NMS confidence threshold
model.iou = 0.6  # NMS IoU threshold
# model.agnostic = False  # NMS class-agnostic
# model.multi_label = False  # NMS multiple labels per box
# model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
# model.max_det = 1000  # maximum number of detections per image
# model.amp = False  # Automatic Mixed Precision (AMP) inference

YOLOv5 🚀 2022-11-24 Python-3.7.15 torch-1.7.1 CUDA:0 (Tesla V100-PCIE-32GB, 32510MiB)

Fusing layers... 
YOLOv5l6-custom summary: 346 layers, 76187892 parameters, 0 gradients, 110.0 GFLOPs
Adding AutoShape... 


In [7]:
prediction_strings = []
file_names = []

for i in tqdm(range(len(IMAGES))):
    prediction_string = ''
    
    output = model(IMAGES[i], size=INFERENCE_SIZE, augment=AUGMENT)
    for bbox in output.pandas().xyxy[0].values:
        xmin, ymin, xmax, ymax, conf, cls, _ = bbox
        prediction_string += f'{cls} {conf} {xmin} {ymin} {xmax} {ymax} '
    
    path_split = IMAGES[i].split('/')
    image_path = f'{path_split[-2]}/{path_split[-1]}'

    prediction_strings.append(prediction_string)
    file_names.append(image_path)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.head()

100%|██████████| 4871/4871 [12:00<00:00,  6.76it/s]


,PredictionString,image_id
0,7 0.9912117123603821 601.3900756835938 516.462...,test/0000.jpg
1,5 0.9795872569084167 346.3387145996094 252.094...,test/0001.jpg
2,1 0.9759643077850342 775.8338623046875 405.938...,test/0002.jpg
3,9 0.9734921455383301 145.88980102539062 259.78...,test/0003.jpg
4,0 0.9683662056922913 192.58596801757812 248.83...,test/0004.jpg


In [8]:
submission

,PredictionString,image_id
0,7 0.9912117123603821 601.3900756835938 516.462...,test/0000.jpg
1,5 0.9795872569084167 346.3387145996094 252.094...,test/0001.jpg
2,1 0.9759643077850342 775.8338623046875 405.938...,test/0002.jpg
3,9 0.9734921455383301 145.88980102539062 259.78...,test/0003.jpg
4,0 0.9683662056922913 192.58596801757812 248.83...,test/0004.jpg
...,...,...
4866,1 0.9835401177406311 292.7864685058594 345.664...,test/4866.jpg
4867,4 0.8317476511001587 416.8661193847656 312.013...,test/4867.jpg
4868,7 0.9793619513511658 117.04243469238281 445.82...,test/4868.jpg
4869,1 0.971454381942749 838.19873046875 0.65328371...,test/4869.jpg


In [9]:
submission.to_csv(f'results/result_{name}.csv', index = None)